In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.high_gamma import load_file, create_inputs
from reversible2.high_gamma import load_train_test
th.backends.cudnn.benchmark = True
from reversible2.models import deep_invertible


In [ ]:
rng = RandomState(201904113)#2 ganz gut # 13 sehr gut)

X = rng.randn(5,2) * np.array([1,0])[None] + np.array([-1,0])[None]
X = np.sort(X, axis=0)

X_test = rng.randn(500,2) * np.array([1,0])[None] + np.array([-1,0])[None]
X_test = np.sort(X_test, axis=0)
train_inputs = np_to_var(X[::2], dtype=np.float32)
val_inputs = np_to_var(X[1::2], dtype=np.float32)
test_inputs = np_to_var(X_test, dtype=np.float32)

plt.figure(figsize=(5,5))
plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1])
plt.scatter(var_to_np(val_inputs)[:,0], var_to_np(val_inputs)[:,1])
plt.scatter([-1],[0], color='black')
plt.xlim(-2.5,5.5)
plt.ylim(-4,4)


In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-3}])

In [ ]:
n_epochs = 10001

for i_epoch in range(n_epochs):
    nll = -th.mean(model_and_dist.get_total_log_prob(0, train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2))
    optim.zero_grad()
    nll.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        plt.title("Train NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        display_close(fig)
    

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-3}])
optim_stds = th.optim.Adam([{'params': tr_log_stds, 'lr':1e-2},])

In [ ]:
n_epochs = 10001

for i_epoch in range(n_epochs):
    outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        plt.axis('equal') #before not done
        display_close(fig)
        
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)

    

## from start with mixture

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-3},])

In [ ]:
n_epochs = 101

for i_epoch in range(n_epochs):
    outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    va_out = model_and_dist.model(val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2).detach()

    demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
    rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    


### only stds

In [ ]:
n_epochs = 1001

for i_epoch in range(n_epochs):
    
    tr_out = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    va_out = model_and_dist.model(val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2).detach()

    demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
    rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

In [ ]:
tr_log_stds

In [ ]:

optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3,
                      'betas': (0,0.99)},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-3,
                      'betas': (0,0.99)}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-2,
                      'betas': (0,0.99)},])

In [ ]:
n_epochs = 201

for i_epoch in range(n_epochs):
    outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    va_out = model_and_dist.model(val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2).detach()

    demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
    rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Train", "Fake"),)
        display_close(fig)

    

In [ ]:
from reversible2.dist_model import ModelAndDist

In [ ]:
fig = plt.figure(figsize=(5,5))
mean, std = model_and_dist.dist.get_mean_std(0)
plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])


plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

plt.axis('equal')
plt.title("Input space")
plt.legend(("Test", "Train", "Fake"),)
display_close(fig)


In [ ]:
test_inputs.shape

In [ ]:
np.array([1,0])[None] + np.array([-1,0])[None]

In [ ]:
-th.mean(get_gaussian_log_probs(np_to_var([-1,], dtype=np.float32), np_to_var([0,], dtype=np.float32),
                      test_inputs[:,0:1]))

In [ ]:
tr_log_stds

In [ ]:
from reversible2.gaussian import get_gaussian_log_probs

## stop gradient from out for mixture

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    with th.no_grad():
        outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
        outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
        ins = invert(model_and_dist.model, outs).detach()
    outs = model_and_dist.model(ins)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    va_out = model_and_dist.model(val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2).detach()

    demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
    rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    with th.no_grad():
        outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
        outs = outs + (th.randn_like(outs) * th.exp(tr_log_stds))
        ins = invert(model_and_dist.model, outs).detach()
    outs = model_and_dist.model(ins)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
    va_out = model_and_dist.model(val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2).detach()

    this_log_stds = th.mean(tr_log_stds, dim=1).unsqueeze(-1)
    demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
    rescaled = demeaned / th.exp(this_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - this_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

In [ ]:

optim_dist = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    test_ins = test_inputs
    outs = model_and_dist.model(test_ins + (th.rand_like(test_ins) -0.5) * 1e-2)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim_dist.zero_grad()
    nll.backward()
    optim_dist.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

## Train on both train and valid

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    val_and_train_ins = th.cat((train_inputs, val_inputs))
    outs = model_and_dist.model(val_and_train_ins + (th.rand_like(val_and_train_ins) -0.5) * 1e-2)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

### retrain only distribution on test

In [ ]:

optim_dist = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    test_ins = test_inputs
    outs = model_and_dist.model(test_ins + (th.rand_like(test_ins) -0.5) * 1e-2)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim_dist.zero_grad()
    nll.backward()
    optim_dist.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

## mixture in input space

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds], 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    ins = train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2
    ins = ins  + (th.randn_like(ins) * th.exp(tr_log_stds))
    outs = model_and_dist.model(ins)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    demeaned = val_inputs.unsqueeze(1) - train_inputs.unsqueeze(0)
    rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

    eps = 1e-8
    log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
    # sum over dimensions
    log_probs = th.sum(log_probs, dim=-1)
    probs = th.exp(log_probs)
    probs = th.mean(probs, dim=1) 
    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])
        ax = plt.gca()
        for trin, lstd in zip(train_inputs, tr_log_stds):
                ellipse = Ellipse(var_to_np(trin), var_to_np(th.exp(lstd)[0] * 2), var_to_np(th.exp(lstd)[1] * 2),
                                  edgecolor='blue', facecolor='None')
                ax.add_artist(ellipse)
        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        
        display_close(fig)    

In [ ]:
#a) model of likelihood for current distribution

### full covariance matrix for micture

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse
from reversible2.gaussian import get_gauss_samples


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)
tr_log_stds = (th.zeros_like(train_inputs, requires_grad=True) - 2).detach().clone().requires_grad_(True)
tr_logit_corrs = (th.zeros_like(train_inputs[:,0], requires_grad=True)).detach().clone().requires_grad_(True)

from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_stds = th.optim.Adam([{'params': [tr_log_stds, tr_logit_corrs], 'lr':1e-3},])

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    with th.no_grad():
        outs = model_and_dist.model(train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2)
        stds = th.exp(tr_log_stds) 
        corrs = th.sigmoid(tr_logit_corrs)
        covs = th.prod(stds, dim=1) * corrs
        cov_matrices = th.stack([th.diag(s ** 2) + c.repeat(2,2) - th.diag(c.repeat(2)) for s, c in zip(stds, covs)])
        mix_dists = [th.distributions.MultivariateNormal(loc=m, covariance_matrix=c_mat) 
                     for m,c_mat in zip(outs, cov_matrices)]
        samples = th.cat([dist.sample((1,)) for dist in mix_dists])
        ins = invert(model_and_dist.model, samples).detach()
    outs = model_and_dist.model(ins)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs)
    va_out = model_and_dist.model(val_inputs).detach()
    stds = th.exp(tr_log_stds) 
    corrs = th.sigmoid(tr_logit_corrs)
    covs = th.prod(stds, dim=1) * corrs
    cov_matrices = th.stack([th.diag(s ** 2) + c.repeat(2,2) - th.diag(c.repeat(2)) for s, c in zip(stds, covs)])

    mix_dists = [th.distributions.MultivariateNormal(loc=m, covariance_matrix=c_mat) 
                 for m,c_mat in zip(tr_out, cov_matrices)]

    probs = th.mean(th.stack([th.exp(dist.log_prob(va_out)) for dist in mix_dists]), dim=0)

    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        stds = th.exp(tr_log_stds) 
        corrs = th.sigmoid(tr_logit_corrs)
        covs = th.prod(stds, dim=1) * corrs
        cov_matrices = th.stack([th.diag(s ** 2) + c.repeat(2,2) - th.diag(c.repeat(2)) for s, c in zip(stds, covs)])

        mix_dists = [th.distributions.MultivariateNormal(loc=m, covariance_matrix=c_mat) 
                     for m,c_mat in zip(tr_out, cov_matrices)]

        probs = th.mean(th.stack([th.exp(dist.log_prob(va_out)) for dist in mix_dists]), dim=0)

        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        for out, lstd in zip(tr_out, tr_log_stds):
            ellipse = Ellipse(var_to_np(out), var_to_np(th.exp(lstd)[0]), var_to_np(th.exp(lstd)[1]),
                              edgecolor='blue', facecolor='None')
            ax.add_artist(ellipse)
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])

        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        display_close(fig)    

In [ ]:
th.prod(th.sqrt(th.diag(cov_matrices[0])))

In [ ]:
cov_matrices

In [ ]:
tr_log_stds

In [ ]:
from reversible2.invert import invert
n_epochs = 10001

for i_epoch in range(n_epochs):
    ins = train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2
    ins = ins  + (th.randn_like(ins) * th.exp(tr_log_stds))
    outs = model_and_dist.model(ins)
    nll = -th.mean(model_and_dist.dist.get_total_log_prob(0, outs))
    optim.zero_grad()
    nll.backward()
    optim.step()
    
    tr_out = model_and_dist.model(train_inputs)
    va_out = model_and_dist.model(val_inputs)
    te_out = model_and_dist.model(test_inputs)
    stds = th.exp(tr_log_stds) 

    corrs = th.sigmoid(tr_logit_corrs)

    covs = th.prod(stds, dim=1) * corrs

    cov_matrices = th.stack([th.diag(s ** 2) + c.repeat(2,2) - th.diag(c.repeat(2)) for s, c in zip(stds, covs)])

    mix_dists = [th.distributions.MultivariateNormal(loc=m, covariance_matrix=c_mat) 
                 for m,c_mat in zip(tr_out, cov_matrices)]

    probs = th.mean(th.stack([th.exp(dist.log_prob(va_out)) for dist in mix_dists]), dim=0)

    nll = -th.mean(th.log(probs + eps))
    #optim.zero_grad()
    optim_stds.zero_grad()
    nll.backward()
    #optim.step()
    optim_stds.step()
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)


        demeaned = va_out.unsqueeze(1) - tr_out.unsqueeze(0)
        rescaled = demeaned / th.exp(tr_log_stds).unsqueeze(0)

        eps = 1e-8
        log_probs = -(rescaled **2) / 2 - np.log(float(np.sqrt(2 * np.pi))) - tr_log_stds.unsqueeze(0)
        # sum over dimensions
        log_probs = th.sum(log_probs, dim=-1)
        probs = th.exp(log_probs)
        probs = th.mean(probs, dim=1) 
        log_probs = th.log(probs + eps)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        for lprob, out in zip(log_probs, va_out):
            plt.annotate("{:.1E}".format(-lprob.item()), var_to_np(out))
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0]), var_to_np(std[1]), edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}\n"
                  "ValidMixNLL {:.1E}\n".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
            -th.mean(log_probs).item(),
        ))
        plt.axis('equal')
        display_close(fig)
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])
        ax = plt.gca()
        for trin, lstd in zip(train_inputs, tr_log_stds):
                ellipse = Ellipse(var_to_np(trin), var_to_np(th.exp(lstd)[0] * 2), var_to_np(th.exp(lstd)[1] * 2),
                                  edgecolor='blue', facecolor='None')
                ax.add_artist(ellipse)
        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        
        display_close(fig)    

## weight norm idea

In [ ]:
cuda = False
from reversible2.distribution import TwoClassIndependentDist

from reversible2.blocks import dense_add_block
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter
from matplotlib.patches import Ellipse


set_random_seeds(2019011641, cuda)
model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
    dense_add_block(2,200),
)


dist = TwoClassIndependentDist(2, truncate_to=None)

from reversible2.dist_model import set_dist_to_empirical
from reversible2.dist_model import ModelAndDist

model_and_dist = ModelAndDist(model, dist)
optim_model = th.optim.Adam([
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
optim_dist = th.optim.Adam([{'params': model_and_dist.dist.parameters(), 'lr':1e-3},
                     ])

from reversible2.weight_norm import weight_norm

all_w_norms = []
for m in model.modules():
    if hasattr(m, 'weight'):
        w_norm = th.zeros(1, requires_grad=True)
        weight_norm(m, fixed_log_norm=w_norm)
        all_w_norms.append(w_norm)
        

optim_wnorm = th.optim.Adam([{'params': all_w_norms, 'lr':1e-3},])


for w in all_w_norms: 
    w.data[:] = -1
set_dist_to_empirical(model_and_dist.model,model_and_dist.dist, [train_inputs])

In [ ]:
n_epochs = 10001

for i_epoch in range(n_epochs):
    nll = -th.mean(model_and_dist.get_total_log_prob(0, train_inputs + (th.rand_like(train_inputs) -0.5) * 1e-2))
    optim_model.zero_grad()
    optim_dist.zero_grad()
    nll.backward()
    optim_model.step()
    optim_dist.step()
    nll = -th.mean(model_and_dist.get_total_log_prob(0, val_inputs + (th.rand_like(val_inputs) -0.5) * 1e-2))
    optim_wnorm.zero_grad()
    optim_dist.zero_grad()
    nll.backward()
    optim_wnorm.step()
    optim_dist.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        tr_out = model_and_dist.model(train_inputs)
        va_out = model_and_dist.model(val_inputs)
        te_out = model_and_dist.model(test_inputs)

        fig = plt.figure(figsize=(5,5))
        mean, std = model_and_dist.dist.get_mean_std(0)
        plt.plot(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1], color=seaborn.color_palette()[1])

        plt.scatter(var_to_np(tr_out)[:,0], var_to_np(tr_out)[:,1], color=seaborn.color_palette()[0])
        plt.scatter(var_to_np(va_out)[:,0], var_to_np(va_out)[:,1], color=seaborn.color_palette()[2])
        ellipse = Ellipse(var_to_np(mean), var_to_np(std[0] * 2), var_to_np(std[1] * 2),
                          edgecolor='black', facecolor='None')
        ax = plt.gca()
        ax.add_artist(ellipse)
        plt.axis('equal')
        plt.title("Epoch {:d} of {:d}\nTrain NLL {:.1E}\nValid NLL {:.1E}\nTest NLL {:.1E}".format(
            i_epoch, n_epochs,
            -th.mean(model_and_dist.get_total_log_prob(0, train_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, val_inputs)).item(),
            -th.mean(model_and_dist.get_total_log_prob(0, test_inputs)).item(),
        ))
        display_close(fig)
    
        examples = model_and_dist.get_examples(0,200)
        fig = plt.figure(figsize=(5,5))
        plt.plot(var_to_np(test_inputs)[:,0], var_to_np(test_inputs)[:,1], color=seaborn.color_palette()[2])
        plt.scatter(var_to_np(examples)[:,0], var_to_np(examples)[:,1], color=seaborn.color_palette()[1])
        plt.scatter(var_to_np(train_inputs)[:,0], var_to_np(train_inputs)[:,1], color=seaborn.color_palette()[0])
        plt.axis('equal')
        plt.title("Input space")
        plt.legend(("Test", "Fake","Train", ),)
        
        display_close(fig)    

In [ ]:
te_in = test_inputs.unsqueeze(1)
noise = th.rand(len(te_in), 200, 2) - 0.5
te_in = te_in + noise * 0.1
te_in = te_in.view(-1, te_in.shape[-1])
plt.scatter(var_to_np(te_in)[:,0], var_to_np(te_in)[:,1])
te_out = model_and_dist.model(te_in)
plt.figure()

plt.scatter(var_to_np(te_out)[:,0], var_to_np(te_out)[:,1])

#plt.axis('equal')

In [ ]:
te_in